# South Dakota

In this notebook the enrollment files for South Dakota HS enrollment are downloaded. The enrollment files contain the file upload date so the download links will need to be found by a partitial text search.

First read the packages needed to acquire the enrollment files.

In [1]:
import datetime
import time
import wget
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import zipfile
from geopy.geocoders import Nominatim

Below an object to find the current file name is located.

In [108]:
# HS enrollment file Name
PreYearText = "http://doe.sd.gov/ofm/documents/Pschgen"
Year = str(int(datetime.datetime.now().year - 1))[2:]
extention = ".xlsx"
File = PreYearText + Year + extention

Using the file url download link created above the file can be downloaded.

In [110]:
FileName = wget.download(File)

100% [............................................................................] 123361 / 123361

Using Pandas, the downloaded file can be read into memory.

In [111]:
SD = pd.read_excel(FileName, skiprows = 2)

The first few rows of the downloaded file are displayed below.

In [112]:
SD.head()

District No.                   District Name  School No.  \
0        1001.0  Plankinton 01-1                        1.0   
1        1001.0  Plankinton 01-1                        2.0   
2        1001.0  Plankinton 01-1                        3.0   
3        1001.0  Plankinton 01-1                        4.0   
4        1001.0  Plankinton 01-1                        5.0   

                                School Name    PK    KG    01    02    03  \
0  Plankinton High School                     0.0   0.0   0.0   0.0   0.0   
1  Plankinton Elementary                     20.0  25.0  17.0  21.0  19.0   
2  Plankinton Junior High                     0.0   0.0   0.0   0.0   0.0   
3  Aurora Plains Academy Middle School        0.0   0.0   0.0   0.0   0.0   
4  Aurora Plains Academy High School          0.0   0.0   0.0   0.0   0.0   

     04    05    06    07    08    09    10    11    12  TOTAL KG-12  \
0   0.0   0.0   0.0   0.0   0.0  22.0  14.0  27.0  12.0         75.0   
1  22.0  19.0  25.0   0.0   0.0   0.0   0.0   0.0   0.0        148.0   
2   0.0   0.0   0.0  14.0  23.0   0.0   0.0   0.0   0.0         37.0   
3   0.0   2.0   6.0   6.0  11.0   0.0   0.0   0.0   0.0         25.0   
4   0.0   0.0   0.0   0.0   0.0  14.0  16.0  10.0   5.0         45.0   

   TOTAL PK-12  
0         75.0  
1        168.0  
2         37.0  
3         25.0  
4         45.0

Next the enrollment file for this year is subset to juniors and seniors and reformatted. This year the file is broken out by gender whereas last years file was not. The last step in the cell below is grouped and summed to aggregate to total enrollment per class/HS school.

In [114]:
SD = SD[['District No.', 'District Name', 'School No.', 'School Name','11', '12']]
SD = SD.rename(columns = {'11': 'Juniors This Year',
                    '12': 'Seniors This Year'})
SD = SD[(SD['Juniors This Year'] > 0) | (SD['Seniors This Year'] > 0)]

SD = SD.groupby(['District No.', 
                         'District Name', 
                         'School No.', 
                         'School Name'], as_index=False).sum()

Next the file name for last years enrollment is created into the object below.

In [116]:
LastYear = str(int(datetime.datetime.now().year - 2))[2:]
File = PreYearText + LastYear + extention
File

'http://doe.sd.gov/ofm/documents/Pschgen16.xlsx'

Using the object for the filename created above is downloaded via wget.

In [117]:
FileName = wget.download(File)

100% [............................................................................] 151587 / 151587

After the file from last year is acquired it can be read into memory via Pandas.

In [118]:
LsYrSD = pd.read_excel(FileName, skiprows = 2)
LsYrSD.head()

District No.                   District Name  School No.  \
0        1001.0  Plankinton 01-1                        1.0   
1        1001.0  Plankinton 01-1                        1.0   
2        1001.0  Plankinton 01-1                        2.0   
3        1001.0  Plankinton 01-1                        2.0   
4        1001.0  Plankinton 01-1                        3.0   

                                School Name  Gender    PK    KG    01    02  \
0  Plankinton High School                    Female   0.0   0.0   0.0   0.0   
1  Plankinton High School                    Male     0.0   0.0   0.0   0.0   
2  Plankinton Elementary                     Female  14.0   4.0  10.0   7.0   
3  Plankinton Elementary                     Male     9.0  10.0  13.0  16.0   
4  Plankinton Junior High                    Female   0.0   0.0   0.0   0.0   

     03     ...         05   06    07    08    09    10   11    12  \
0   0.0     ...        0.0  0.0   0.0   0.0   7.0   9.0  7.0  11.0   
1   0.0     ...        0.0  0.0   0.0   0.0  10.0  19.0  6.0  11.0   
2  11.0     ...       15.0  7.0   0.0   0.0   0.0   0.0  0.0   0.0   
3  13.0     ...       10.0  8.0   0.0   0.0   0.0   0.0  0.0   0.0   
4   0.0     ...        0.0  0.0  12.0  12.0   0.0   0.0  0.0   0.0   

   TOTAL KG-12  TOTAL PK-12  
0         34.0         34.0  
1         46.0         46.0  
2         66.0         80.0  
3         82.0         91.0  
4         24.0         24.0  

[5 rows x 21 columns]

Next the file for last year is subset and reformatted.

In [119]:
LsYrSD = LsYrSD[['District No.', 'District Name', 'School No.', 'School Name','11', '12']]
LsYrSD = LsYrSD.rename(columns = {'11': 'Juniors Last Year',
                    '12': 'Seniors Last Year'})
LsYrSD = LsYrSD[(LsYrSD['Juniors Last Year'] > 0) | (LsYrSD['Seniors Last Year'] > 0)]

LsYrSD = LsYrSD.groupby(['District No.', 
                         'District Name', 
                         'School No.', 
                         'School Name'], as_index=False).sum()

Below the enrollment files for this year and last year are joined together.

In [122]:
SD = pd.merge(SD, LsYrSD, 
              how = 'inner', 
              on = ['District No.', 
                    'District Name', 
                    'School No.', 
                    'School Name'])

After the 2 files are joined together, the enrollment deltas are calculated.

In [ ]:
SD['JuniorDelta'] = SD['Juniors This Year']/SD['Juniors Last Year']
SD['SeniorDelta'] = SD['Seniors This Year']/SD['Seniors Last Year']

Next the hs lookup files is downloaded. For this Selenium is used, the files contain the upload date, so the file needs to be found via text search.

In [87]:
# Set the preferences for the firefox web browser
fp = webdriver.FirefoxProfile()
fp.set_preference('browser.download.folderList', 2)
fp.set_preference('browser.download.manager.showWhenStarting', False)
fp.set_preference('browser.download.dir', '/tmp')
fp.set_preference("http.response.timeout", 300)
fp.set_preference("dom.max_script_run_time", 300)
fp.set_preference('webdriver.load.strategy', 'unstable')
fp.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/plain, application/vnd.ms-excel, text/csv, text/comma-separated-values, application/octet-stream, application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
fp.update_preferences()

Launch the browser and navigate to the education directory site.

In [48]:
# MDE web app for hs enrollment data
searchURL = 'http://doe.sd.gov/ofm/edudir.aspx'
time.sleep(3)

# Launch web browser and navigate to the searchURL
driver = webdriver.Firefox(fp)
driver.get(searchURL)
time.sleep(3)

Find the lookup file via Selenium.

In [49]:
ids_a =  driver.find_elements_by_xpath("//*[text()='Address List: Principal & School Info. (Excel)']")
for ii in ids_a:
    link = ii.get_attribute('href')
time.sleep(4)

Once the link has been found, the file can be downloaded with wget.

In [53]:
FileName = wget.download(link)

100% [............................................................................] 153135 / 153135

After the lookup file is downloaded it can be read into memory via Pandas.

In [55]:
SDLookUp = pd.read_excel(FileName)

Subset the lookup dataframe to the columns of interest below.

In [57]:
SDLookUp = SDLookUp[['DistrictType', 'SchoolType', 'DistrictNumber', 'DistrictName',
       'SchoolNumber', 'SchoolName', 'SchoolAddress', 'SchoolCity', 'SchoolState',
       'SchoolZip']]

Below the enrollment file and lookup file are joined together.

In [96]:
SDFinal = pd.merge(SDLookUp, SD, how = 'inner',
        left_on = ['DistrictNumber', 
       'SchoolNumber'],
        right_on = ['District No.', 
                    'School No.'])

Finally the cleaned South Dakota file can be written to a csv file.

In [70]:
SDFinal.to_csv('SDFinalUpdatedZip.csv', index = False)